# Lab for loading and transforming the data
Responsible: Fabian (01126693)

## Task

As group 5 we have chosen task 6. Task 6 deals with natural disasters and poses 3 questions to be answered:

1. How did the number of deaths per year from natural disasters change over the last hundred years? 
2. How does this vary by country? How does this vary by type of natural disaster?  
3. Are there trends visible that could be due to climate change? 

Based on the questions posed, some requirements for the data set emerge. Question 1 aims at the change in death counts over the last 100 years. Therefore, the dataset must contain death counts as well as a history as long as possible (optimally in the range of 100 years). Question 2 focuses on the development in different countries and whether there are differences between different types of natural disasters. Therefore, the data set must include both geographic characteristics and attributes to distinguish between different types of natural disasters. 

## Data Source(s)
Based on these requirements, publicly available datasets were sought. The 2 most relevant of these are presented briefly here and an explanation is given of how the final choice of dataset was made. 

### WHO data [<sup>1</sup>](#fn1) ( Explored but not used)

The data was queried from the Mortality Database provided by the WHO. With regard to the origin, completeness and scope of the data, the WHO says: 

> The WHO Mortality Database is a compilation of mortality data by country and area, year, sex, age and cause of death, as transmitted annually by national authorities from their civil registration and vital statistics system. It comprises data since 1950 to date. Only data with at least 65% completeness are published here

##### Requirements met
 - Total death counts per year 
 - Split by country and geographical region 
 - Split by sex (additional information)
 - Split by age (additional information)

##### Requirements not met
 - Long history missing: data for natural disasters starts 1979
 - Missing split by type of natural disaster
 
It can be seen that the WHO data are more detailed in terms of gender and age categories, but they have no distinction of natural disaster categories and also have a limited history of only 50 years. 

### EM-DAT data [<sup>2</sup>](#fn2) (Explored and used)

> In 1988, the Centre for Research on the Epidemiology of Disasters (CRED) launched the Emergency  Events Database (EM-DAT). EM-DAT was created with the initial support of the World Health Organisation (WHO) and the Belgian Government.
>
>The main objective of the database is to serve the purposes of humanitarian action at national and international levels. The initiative aims to rationalise decision making for disaster preparedness, as well as provide an objective base for vulnerability assessment and priority setting.
>
>EM-DAT contains essential core data on the occurrence and effects of over 22,000 mass disasters in the world from 1900 to the present day. The database is compiled from various sources, including UN agencies, non-governmental organisations, insurance companies, research institutes and press agencies.
>
> -- Source: https://www.emdat.be/

##### Requirements met
 - Total death counts per event 
 - Country, region and continent information per event
 - Events dated back till 1900
 - Disaster type per event
 
Since the questions do not require an analysis based on gender or age categories, but the EM-DAT dataset covers a much longer period (since 1900) and also assigns a category to each natural disaster, this dataset is perfectly suited for the task and therefore used for the analysis. In the following section, a detailed overview of the attributes used in the further analysis will be given. 

#### Data fields

 - `Disaster_Group`: Two different types of disasters can be distinguished in the EM-DAT database: natural disasters and technological disasters. Since we are only interested in natural disasters, only they were requested and consequently this field contains only the value *Natural*. No missing values are present for this attribute. 
 - `Disaster_Subgroup`: Every natural disaster is assigned to one of the following six subgroups: *Biological, Geophysical, Climatological, Hydrological, Meteorological and Extra-terrestrial* to describe the type of natural disaster. No missing values are present for this attribute. 
 - `Disaster_type`: For every natural disaster event one main disaster type is identified. If two or more disasters are related because they are consequences of each other, then this information is encoded in the attributes `Associated_Dis` and `Associated_Dis2`. No missing values are present for this attribute. 
 
 - `Disaster Sub-Type`: Subdivision related to the attribute `Disaster_type` so that a the disaster type Storm can be further classified as tropical, extra-tropical or convective storm. 
 
 - `Disaster Sub-Sub Type`: Any appropriate sub-division of the disaster sub-type (not applicable for all disaster sub-types).
 
> Types of natural disasters could be further broken down using two more categories which would be available in the database. For example, the Disaster type *Storm* could be further subdivided into *Tropical storm*, *Extra-tropical storm* or *Convective storm*. Even a further subdivision of the category *Convective storm* would be possible. Since the analysis is aimed at detecting trends on a high level, the classification of each event based on the attributes `Disaster_Subgroup` and `Disaster _Type` was considered sufficient and the further subdivisions into `Disaster sub-type` and `Disaster Subsubtype` is only intended to be considered for detailed analysis. The full table is saved in /data/disaster_classification (w/o the technological disaster group) and shown in the Appendix of this notebook.

 - `Associated_Dis`: Secondary event triggered by a natural disaster (i.e. Landslide for a flood, explosion after an earthquake, ...)
 - `Associated_Dis2`: Another secondary event triggered by a natural disaster. (i.e. Landslide for a flood, explosion after an earthquake, ...)
 
 Example: If a tsunami is triggered by an earthquake, then the attribute `Disaster_Type` would be *Earthquake*, the attribute `Disaster_Subtype` would be *Ground movement* and the attribute `Associated_Dis` would be *Tsunami/Tidal wave*.
 
 - `Country`: The country in which the disaster has occurred or had an impact. If a disaster has affected more than one country, a seperate entry is created in the database for each country affected. No missing values are present for this attribute. 
 - `ISO`: Unique 3-letter code for each country defined by ISO 3166. No missing values are present for this attribute. 
 - `Region`: The region to which the country belongs, based on the UN regional division. No missing values are present for this attribute. 
 - `Continent`: The continent to which the country belongs. No missing values are present for this attribute. 
 - `Start_Year`: The year when the disaster occurred. No missing values are present for this attribute.
 
 - `End Year`: The year when the disaster ended. No missing values are present for this attribute.
 
> For sudden-impact disasters also the month and the day are well defined and available. For disaster situations developing gradually over a longer time period (i.e. drought) with no specific start date the day attribute is empty. For our questions the exact date plays a subordinate role and therefore the year of the beginning of the disaster is completely sufficient for our analysis.    

 - `Total_Deaths`: Number of people who lost their life because the event happened plus the number of people whose whereabouts since the disaster are unknown, and presumed dead based on official figures. Missing values present for approx. 25% of all events.
 
 - `No_Affected`: Number of people which requiring immediate assistance during an emergency situation. The indicator affected is often reported and is widely used by different actors to convey the extent, impact, or severity of a disaster in non-spatial terms. In case that no values for the attribute `Total_Deaths` are available this attribute could be used as a proxy. 

### Load - Transform - Explore - Export

The next steps are to load the data, assign a corresponding data type to each column, and give a brief overview of the data. The detailed analysis of the data set happens in the course of the answering of the questions defined above further down in the notebook. At the end, the data set should be saved as a pickle for further processing. 

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
raw_xlsx = pd.read_excel("../data/emdat.xlsx",
                         skiprows = 5,
                         header = 1,
                         usecols = ['Disaster Group',
                                    'Disaster Subgroup',
                                    'Disaster Type',
                                    'Disaster Subtype',
                                    'Disaster Subsubtype',
                                    'Associated Dis',
                                    'Associated Dis2',
                                    'Country',
                                    'ISO',
                                    'Region',
                                    'Continent',
                                    'Start Year',
                                    'End Year',
                                    'Total Deaths',
                                    'Total Affected'])
raw_xlsx.columns = raw_xlsx.columns.str.replace(' ', '_')
raw_xlsx.head()

C:\Users\Koffe\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Disaster_Group,Disaster_Subgroup,Disaster_Type,Disaster_Subtype,Disaster_Subsubtype,Country,ISO,Region,Continent,Associated_Dis,Associated_Dis2,Start_Year,End_Year,Total_Deaths,Total_Affected
0,Natural,Climatological,Drought,Drought,NaN,Cabo Verde,CPV,Western Africa,Africa,Famine,NaN,1900,1900,11000.0,NaN
1,Natural,Climatological,Drought,Drought,NaN,India,IND,Southern Asia,Asia,NaN,NaN,1900,1900,1250000.0,NaN
2,Natural,Geophysical,Earthquake,Ground movement,NaN,Guatemala,GTM,Central America,Americas,Tsunami/Tidal wave,NaN,1902,1902,2000.0,NaN
3,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Guatemala,GTM,Central America,Americas,NaN,NaN,1902,1902,1000.0,NaN
4,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Guatemala,GTM,Central America,Americas,NaN,NaN,1902,1902,6000.0,NaN


In [3]:
df_transformed = pd.DataFrame() 
df_transformed = raw_xlsx.convert_dtypes()
df_transformed.dtypes

Disaster_Group         string
Disaster_Subgroup      string
Disaster_Type          string
Disaster_Subtype       string
Disaster_Subsubtype    string
Country                string
ISO                    string
Region                 string
Continent              string
Associated_Dis         string
Associated_Dis2        string
Start_Year              Int64
End_Year                Int64
Total_Deaths            Int64
Total_Affected          Int64
dtype: object

In [4]:
print(df_transformed.shape[0])
print(df_transformed['Total_Deaths'].isna().sum() )
print(df_transformed['Start_Year'].min())
print(df_transformed['Start_Year'].max())

df_transformed['Disaster_Decade'] = df_transformed['Start_Year']//10*10 
print(df_transformed["Total_Deaths"].isnull().groupby(df_transformed.Disaster_Decade).sum().astype(int))
print(df_transformed["Total_Deaths"].isnull().groupby(df_transformed.Continent).sum().astype(int))

16488
4748
1900
2022
Disaster_Decade
1900       8
1910      18
1920      20
1930      16
1940      28
1950      26
1960     142
1970     256
1980     613
1990     897
2000    1301
2010    1033
2020     390
Name: Total_Deaths, dtype: int32
Continent
Africa       969
Americas    1334
Asia        1280
Europe       810
Oceania      355
Name: Total_Deaths, dtype: int32


#### Missing data

As discussed earlier, data is collected from multiple sources and therefore there is always the possibility of missing values. The data set contains a total of 16488 recorded events starting from 1900 up to 2022. For the attribute `Total Deaths` it can be seen that 4748 records have a missing value which corresponds to a portion of approximately 29% of missing values. Due to the nature of the available data, interpolation of missing values is not appropriate, as disasters can vary significantly in severity, location, and number of people killed. One way to approximate the missing data in terms of deaths is to use the number of affected. However, it must be taken into account that this number is sometimes considerably different from the number of deaths and always overestimates them. Looking at the distribtuion of missing values per decade one can see that the majority of records is from the 1980s or later. When it comes to geografical location of the records with missing data one can observe that the majority of disasters are linked to Americas and Asia. 

The transformed dataframe is serialized with pickle, because in that way also the data types will be preserved for the next steps of the analyis.

In [5]:
transformed_df_serial_path = "../data/data_transformed.pickle"

# some sanity checks
assert isinstance(df_transformed, pd.DataFrame), "\"df_transformed\" is not a pandas dataframe!"

assert len(df_transformed)!=0, "\"df_transformed\" contains no data!"

# serializing the dataframe object to a binary file
with open(transformed_df_serial_path, 'wb') as file:
    pickle.dump(df_transformed, file)

<sup>1</sup> WHO data: <span id="fn1"> https://platform.who.int/mortality/themes/theme-details/topics/indicator-groups/indicator-group-details/MDB/natural-disasters</span>

<sup>2</sup> EM-dat data: <span id="fn2"> https://public.emdat.be/data Based on the terms of use, the data can be used for this exercise:  
    
> If you are an academic organization, a university, a non-profit research institution and/or an international public organization (UN agencies, multi-lateral banks, other multilateral institution and national governments) and/or part of a Media agency (journalist, press agencies) with the intention to use the EM-DAT database (hereafter ‘EM-DAT’) for research, teaching or information purposes, you shall, conditional upon the acceptance of the present conditions of use, be granted free access to EM-DAT (also ‘Authorized Use’).

Filter and dropping of columns in pandas. delete columns 
Specifiy which data is downloaded from EM data
table with categories also in the notebook
add Sub-Type and subsubßType to data 
Explicitly stasting that data is missing


## Temperature Data

from: [https://berkeleyearth.lbl.gov/country-list/]()

In [27]:
country_temp_delta = pd.read_csv("../data/country_temp_delta.csv")
country_temp_delta

,Country,Warming since 1960 (°C / century),Region
0,Afghanistan,3.32 ± 0.34,Asia
1,Åland,3.01 ± 0.24,Europe
2,Albania,1.97 ± 0.28,Europe
3,Algeria,2.86 ± 0.28,Africa
4,American Samoa,1.43 ± 0.57,NaN
...,...,...,...
232,Virgin Islands,1.88 ± 0.30,NaN
233,Western Sahara,2.49 ± 0.69,Africa
234,Yemen,2.50 ± 0.55,Asia
235,Zambia,1.77 ± 0.27,Africa


The "Warming since..." column needs to be split into a warming and uncertainty column.

In [28]:
country_temp_delta[["Warming/Century", "Uncertainty"]] = country_temp_delta["Warming since 1960 (°C / century)"].str.split("±", expand=True).apply(pd.to_numeric, errors="coerce")
country_temp_delta.drop("Warming since 1960 (°C / century)", inplace=True, axis=1)
country_temp_delta

,Country,Region,Warming/Century,Uncertainty
0,Afghanistan,Asia,3.32,0.34
1,Åland,Europe,3.01,0.24
2,Albania,Europe,1.97,0.28
3,Algeria,Africa,2.86,0.28
4,American Samoa,NaN,1.43,0.57
...,...,...,...,...
232,Virgin Islands,NaN,1.88,0.30
233,Western Sahara,Africa,2.49,0.69
234,Yemen,Asia,2.50,0.55
235,Zambia,Africa,1.77,0.27


## Appendix

### Disaster Classification according to EM-DAT

Sourced from https://public.emdat.be/about.

<table style="border: 1px solid lightgray;float:left;">
    <tbody>
        <tr>
            <td>
                <p><strong>Disaster </strong></p>
                <p><strong>Group</strong></p>
            </td>
            <td>
                <p><strong>Disaster </strong></p>
                <p><strong>Sub-Group</strong></p>
            </td>
            <td>
                <p><strong>Disaster </strong></p>
                <p><strong>Type</strong></p>
            </td>
            <td>
                <p><strong>Disaster </strong></p>
                <p><strong>Sub-Type</strong></p>
            </td>
            <td>
                <p><strong>Disaster </strong></p>
                <p><strong>Sub-Sub Type</strong></p>
            </td>
        </tr>
        <tr>
            <td>
                <p><strong>Natural</strong></p>
            </td>
            <td>
                <p>Geophysical</p>
            </td>
            <td>
                <p>Earthquake</p>
            </td>
            <td>
                <p>Ground movement</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Tsunami</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Volcanic activity</p>
            </td>
            <td>
                <p>Ash fall</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Lahar</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Pyroclastic flow</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Lava flow</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Mass Movement</p>
            </td>
            <td></td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>
                <p>Meteorological</p>
            </td>
            <td>
                <p>Storm</p>
            </td>
            <td>
                <p>Tropical storm</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Extra-tropical storm</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Convective storm</p>
            </td>
            <td>
                <p>Derecho</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Hail</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Lightning/thunderstorm</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Rain</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Tornado</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Sand/dust storm</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Winter storm/blizzard</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Storm/surge</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Wind</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Severe Storm</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Extreme Temperature</p>
            </td>
            <td>
                <p>Cold wave</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Heat Wave</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Severe winter conditions</p>
            </td>
            <td>
                <p>Snow/ice</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Frost/freeze</p>
            </td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Fog</p>
            </td>
            <td></td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>
                <p>Hydrological</p>
            </td>
            <td>
                <p>Flood</p>
            </td>
            <td>
                <p>Coastal flood</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Riverine flood</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Flash flood</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Ice jam flood</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Landslide</p>
            </td>
            <td>
                <p>Avalanche (snow, debris, mudflow, rock fall)</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Wave action</p>
            </td>
            <td>
                <p>Rogue wave</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Seiche</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>
                <p>Climatological</p>
            </td>
            <td>
                <p>Drought</p>
            </td>
            <td>
                <p>Drought</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Glacial Lake outburst</p>
            </td>
            <td></td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Wildfire</p>
            </td>
            <td>
                <p>Forest fires</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Land fire: Brush, bush, pasture</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>
                <p>Biological</p>
            </td>
            <td>
                <p>Epidemic</p>
            </td>
            <td>
                <p>Viral diseases</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Bacterial diseases</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Parasitic diseases</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Fungal diseases</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Prion diseases</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Insect Infestation</p>
            </td>
            <td>
                <p>Locust</p>
                <p>Grasshopper</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Animal accident</p>
            </td>
            <td></td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td>
                <p>Extra-terrestrial</p>
            </td>
            <td>
                <p>Impact</p>
            </td>
            <td>
                <p>Airburst</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td>
                <p>Space weather</p>
            </td>
            <td>
                <p>Energic particles</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Geomagnetic storm</p>
            </td>
            <td></td>
        </tr>
        <tr>
            <td></td>
            <td></td>
            <td></td>
            <td>
                <p>Shockwave</p>
            </td>
            <td></td>
        </tr>
    </tbody>
</table>